In [51]:
import numpy as np
import cv2 as cv

In [52]:
def flatten_image_vectors(image):
    """
    Takes in the image as np array of size 256x256 pixels and returns an np array of sixe 1 x (256x256)
    """
    return image.flatten()


In [53]:
def stack_vectors(images):
    stack = []
    for img in images:
        stack.append(flatten_image_vectors(img))

    return np.array(stack)

In [54]:
#Test images randomly generated
images = []

for var in range(200):
    img = [int(np.random.random()*256) for _ in range(65536)]
    images.append(np.array(img))

images = np.array(images)


In [57]:
def compute_mean_vector(stack):
    mean_vector = []
    for var in range(stack.shape[0]):
        mean = np.mean(stack[var, :])
        mean_vector.append(np.array([mean]))
    
    return np.array(mean_vector)

In [58]:
mean_vector = compute_mean_vector(images)

In [59]:
def compute_scatter_matrix(stack, mean_vector):
    scatter_matrix = np.zeros((mean_vector.shape[0],mean_vector.shape[0]))
    for var in range(stack.shape[1]):
        scatter_matrix += (stack[:,var].reshape(mean_vector.shape[0],1) - mean_vector).dot((stack[:,var].reshape(mean_vector.shape[0],1) - mean_vector).T)

    return scatter_matrix
sm = compute_scatter_matrix(images, mean_vector)

In [60]:
sm.shape

(200, 200)

In [61]:
images.shape

(200, 65536)

In [65]:
def compute_covariance_matrix(stack):
    cov_matrix = []
    for var in range(stack.shape[0]):
        cov_matrix.append(stack[var, :])

    return np.cov(np.array(cov_matrix))

In [66]:
cov_matrix = compute_covariance_matrix(images)

In [68]:
cov_matrix.shape

(200, 200)

In [69]:
# eigenvectors and eigenvalues for the from the scatter matrix
eig_val_sc, eig_vec_sc = np.linalg.eig(sm)

# eigenvectors and eigenvalues for the from the covariance matrix
eig_val_cov, eig_vec_cov = np.linalg.eig(cov_matrix)

In [70]:
eig_vec_cov.shape

(200, 200)

In [71]:
def sort_eigenvec_by_eigenval(eigenvals, eigenvecs):
    """
    1. Make a list of (eigenvalue, eigenvector) tuples
    2. Sort the (eigenvalue, eigenvector) tuples from high to low
    """
    eig_pairs = [(np.abs(eigenvals[i]), eigenvecs[:,i]) for i in range(len(eigenvals))]
    eig_pairs.sort(key=lambda x: x[0], reverse=True)

    return  eig_pairs

In [72]:
eig_pairs = sort_eigenvec_by_eigenval(eig_val_cov, eig_vec_cov)

In [73]:
for i in eig_pairs:
    print(i[0])

6056.8507988852925
6044.683504246496
6035.655579312887
6029.348357817652
6020.167793816658
5995.458002137928
5992.627758545764
5986.915900947863
5982.076812288458
5972.287211697593
5967.614422708473
5943.718663798641
5937.893988710458
5934.475509451652
5923.445504566268
5920.52348045284
5912.148703099951
5908.411281583248
5901.72858870951
5888.150057072214
5882.288190587142
5869.583398862771
5864.898929938056
5861.486327686288
5856.551568362767
5847.185793407247
5840.78408779823
5835.219188049007
5834.012142943685
5828.878612074441
5821.167485161271
5819.315687041114
5807.092666032648
5794.7603124774905
5791.384754477718
5787.539742429521
5783.949124608151
5770.222662419929
5766.862371774347
5763.66457553855
5757.103880978963
5754.539041737923
5750.972370974286
5746.608089013786
5744.044835567553
5737.285247712902
5734.5505921661315
5721.683681890274
5714.301511771253
5707.957123477654
5696.753268428718
5692.511541029839
5690.502558903799
5681.434951781253
5674.317916083165
5672.617991

In [74]:
eig_pairs[0][1].shape

(200,)

In [112]:
def choose_k_eigenvecs(eig_pairs, k):
    stack_tuple = (eig_pairs[0][1].reshape(eig_pairs[0][1].shape[0],1),)
    for var in range(1,k):
        stack_tuple += (eig_pairs[var][1].reshape(eig_pairs[var][1].shape[0],1),)

    matrix_w = np.hstack(stack_tuple)
    return matrix_w

In [113]:
matrix_w = choose_k_eigenvecs(eig_pairs, 4)

(array([[ 0.08263818],
       [-0.00658433],
       [ 0.02806179],
       [ 0.0521722 ],
       [ 0.01892813],
       [ 0.15661206],
       [ 0.14750011],
       [-0.01024378],
       [ 0.0614106 ],
       [ 0.01142237],
       [ 0.06150147],
       [-0.01379601],
       [-0.07290126],
       [ 0.10012911],
       [-0.07328245],
       [-0.03128602],
       [-0.06878146],
       [ 0.01340971],
       [-0.14772982],
       [ 0.04146798],
       [-0.13431949],
       [-0.01054127],
       [ 0.02263642],
       [-0.11782017],
       [-0.03104287],
       [-0.03850175],
       [ 0.08839277],
       [-0.02429209],
       [-0.06313295],
       [-0.10602143],
       [ 0.03566729],
       [-0.0266363 ],
       [ 0.00708719],
       [ 0.02855741],
       [ 0.14422996],
       [-0.02601836],
       [ 0.04550205],
       [-0.02761111],
       [-0.11281402],
       [ 0.04936267],
       [-0.03268737],
       [ 0.07190912],
       [-0.0068089 ],
       [-0.12886412],
       [ 0.07982085],
       [ 

In [115]:
matrix_w.shape

(200, 4)

In [82]:
import numpy as np

def normalized(a, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2==0] = 1
    return a / np.expand_dims(l2, axis)

img = images[0]
print(img)
print(normalized(img,0))

[ 60 177 211 ... 121  33 181]
[[0.00158757 0.00468335 0.00558297 ... 0.00320161 0.00087317 0.00478918]]
